## Importing libraries 

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array

import cv2
from skimage import io

## Save location to variables

In [ ]:
# save location of folders in variables
train_dir = r'C:\Users\AKSHAT\Desktop\DL_internshala\Final_Assignment_Face Mask Dataset\Train' 
validation_dir = r'C:\Users\AKSHAT\Desktop\DL_internshala\Final_Assignment_Face Mask Dataset\Validation'
test_dir = r'C:\Users\AKSHAT\Desktop\DL_internshala\Final_Assignment_Face Mask Dataset\Test'

## Image datagenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Data Augmentation using image data-generator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

# for rescaling test data matrices values 
test_datagen = ImageDataGenerator(rescale=1./255)

# training dataset
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(128,128),
                                                   batch_size=32,
                                                   class_mode='binary')
# validation dataset
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                       target_size=(128,128),
                                                       batch_size=32,
                                                       class_mode='binary')

## VGG-19 model

In [ ]:
from tensorflow.keras.applications import VGG19

In [ ]:
conv_base = VGG19(weights='imagenet', 
                 include_top=False, 
                 input_shape=(128,128,3))

In [ ]:
conv_base.summary()

## Creating CNN model with base model as VGG-19

In [ ]:
from tensorflow.keras import layers, models

# connect Fully connected Dense layer and ouput layer in front of the conv_base

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
conv_base.trainable = False

In [ ]:
model.summary()

In [ ]:
# compile the model
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(learning_rate=1e-4), # using smaller LR as we need to fine tune our already trained model
             metrics=['acc'])

## Save best-model

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("Face_Mask_Detection_VGG19_Model-{epoch:02d}.h5",save_best_only=True)

## Model history

In [ ]:
model_history = model.fit(train_generator, 
                                   steps_per_epoch=train_generator.samples/train_generator.batch_size,  
                                   epochs=10,
                                   validation_data=validation_generator, 
                                   validation_steps=validation_generator.samples/validation_generator.batch_size,
                                   callbacks=[checkpoint_cb])

## Model performance visualization of validation and training data

In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

## Testing model performance

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(128,128),
                                                 batch_size=20,
                                                 class_mode='binary')
                                                 

In [ ]:
# evaluate model on test data
model.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size)

## Real time face-mask detection using web-cam

In [ ]:
# load the best-save model
load_model = keras.models.load_model('Face_Mask_Detection_VGG19_Model-08.h5')

In [ ]:
# creating labels
labels = {0: 'WithMask', 1: 'WithoutMask'}

# xml file to take only-face as input from the frames captured by web-cam
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    _, frame = cap.read()

    rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    faces = face_classifier.detectMultiScale(rgb)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_rgb = rgb[y:y+h,x:x+w]
        roi_rgb = cv2.resize(roi_rgb,(128,128),interpolation=cv2.INTER_AREA)

#             x = cv2.resize(frame,(128,128))
        if np.sum([roi_rgb]) != 0 :

            x = np.expand_dims(roi_rgb,axis=0) # reshaping to (1,128,128,3)
            x = test_datagen.flow(x)
# #                     x = img_to_array(x)




            prediction = load_model.predict(x[0])
            label = labels[int(np.round(prediction, decimals=0))]
            print(label)
#             label_position = (x,y)
#             cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
#         else:
#             cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

    cv2.imshow('Face Mask Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break